In [30]:
import pandas as pd
import requests
from datetime import datetime
from jsonpath import jsonpath


def get_Concept_stock(block , date: str = None) -> pd.DataFrame:
    """
    https://push2.eastmoney.com/api/qt/clist/get?
    fid=f62&po=1&pz=50&pn=1&np=1&fltt=2&invt=2
    &ut=b2884a393a59ad64002292a3e90d46a5&fs=b:BK0989
    &fields=f12,f14,f2,f3,f62,f184,f66,f69,f72,f75,f78,f81,f84,f87,f204,f205,f124,f1,f13
    获取指定日期涨停股票行情

    Parameters
    ----------
    range: f62:f164:f174 = 今日:5日:10日 输出参数不一样 todo
    date : str
        指定日期，默认为当前日期 格式形如 ``'20220506'``

    Returns
    -------
    DataFrame
        指定日期涨停股票行情

    Fields
    ------
    ``['日期', '股票代码', '股票名称', '涨跌幅', '成交额', '封板资金', '流通市值', '最新价', '换手率','首次封板时间', '最后封板时间', '炸板次数', '连扳数', '所属行业']``
    """
    if date is None:
        date = datetime.today().strftime('%Y%m%d')
    params = (
        ('ut', 'b2884a393a59ad64002292a3e90d46a5'),
        ('pn', '1'),
        ('pz', '500'),
        ('po', '1'),
        ('np', '1'),
        ('fltt', '2'),
        ('invt', '2'),
        ('fields', 'f12,f14,f2,f3,f62,f184,f66,f69,f72,f75,f78,f81,f84,f87,f204,f205,f124,f1,f13'),
        ('fid','f62'),
        ('fs','b:'+block),
    )

    response = requests.get(
        'https://push2.eastmoney.com/api/qt/clist/get', params=params,  verify=False)
    fields = {
        'f2': '最新价',
        'f3': '涨跌幅',
        'f12': '股票代码',
        'f14': '股票名称',
        'f62': '主力资金净流入'
    }
    items = jsonpath(response.json(), '$..diff[:]')
    if not items:
        df = pd.DataFrame(
            columns=['概念板块','日期']+list(fields.values()))
        return df
    df = pd.DataFrame(items)
    
    
    df: pd.DataFrame = df.rename(columns=fields)[fields.values()]
    
    # df = df.loc[(df['主力资金净流入'].str.contains('-') == True)]
    df = df.drop(df[(df.主力资金净流入 == '-')].index)

    df['主力资金净流入'] /= 100000000
    df['概念板块编码'] = block
    
    return df

df1 = get_Concept_stock('BK0989')
df1.to_excel('~/Downloads/20220714.xlsx',index=False)
df1

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'push2.eastmoney.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,最新价,涨跌幅,股票代码,股票名称,主力资金净流入,概念板块编码
0,539.0,7.01,300750,宁德时代,12.458723,BK0989
1,98.38,5.71,300014,亿纬锂能,3.258325,BK0989
2,33.5,5.21,601615,明阳智能,2.917611,BK0989
3,15.59,4.7,002202,金风科技,2.174216,BK0989
4,29.72,3.16,600406,国电南瑞,1.980346,BK0989
...,...,...,...,...,...,...
138,8.64,0.58,002340,格林美,-0.935712,BK0989
139,100.95,0.47,002460,赣锋锂业,-1.025482,BK0989
140,29.45,-1.57,600884,杉杉股份,-1.877273,BK0989
141,15.62,-9.97,002090,金智科技,-3.787168,BK0989


In [21]:
dfd = df1.主力资金净流入.str.isdigit()
df1.dropna()

,最新价,涨跌幅,股票代码,股票名称,主力资金净流入,概念板块编码
0,539.0,7.01,300750,宁德时代,1245872272.0,BK0989
1,98.38,5.71,300014,亿纬锂能,325832512.0,BK0989
2,33.5,5.21,601615,明阳智能,291761056.0,BK0989
3,15.59,4.7,002202,金风科技,217421584.0,BK0989
4,29.72,3.16,600406,国电南瑞,198034557.0,BK0989
...,...,...,...,...,...,...
138,8.64,0.58,002340,格林美,-93571171.0,BK0989
139,100.95,0.47,002460,赣锋锂业,-102548176.0,BK0989
140,29.45,-1.57,600884,杉杉股份,-187727281.0,BK0989
141,15.62,-9.97,002090,金智科技,-378716768.0,BK0989


In [ ]:
import pandas as pd
import requests
from datetime import datetime
from jsonpath import jsonpath


def get_zt_stock_bank(date: str = None) -> pd.DataFrame:
    """
    获取指定日期涨停股票行情

    Parameters
    ----------
    date : str
        指定日期，默认为当前日期 格式形如 ``'20220506'``

    Returns
    -------
    DataFrame
        指定日期涨停股票行情

    Fields
    ------
    ``['日期', '股票代码', '股票名称', '涨跌幅', '成交额', '封板资金', '流通市值', '最新价', '换手率','首次封板时间', '最后封板时间', '炸板次数', '连扳数', '所属行业']``
    """
    if date is None:
        date = datetime.today().strftime('%Y%m%d')
    params = (
        ('ut', 'b2884a393a59ad64002292a3e90d46a5'),
        ('pn', '1'),
        ('pz', '500'),
        ('po', '1'),
        ('np', '1'),
        ('fields', 'f12,f13,f14,f62'),
        ('fid','f62'),
        ('fs','m:90+t:2'),
    )

    response = requests.get(
        'https://push2.eastmoney.com/api/qt/clist/get', params=params,  verify=False)
    fields = {
        'f12': '板块编码',
        'f13': '上市板块',
        'f14': '板块名称',
        'f62': '主力资金净流入'
    }
    items = jsonpath(response.json(), '$..diff[:]')
    if not items:
        df = pd.DataFrame(
            columns=['日期']+list(fields.values())+['统计天数', '涨停次数'])
        return df
    df = pd.DataFrame(items)
    
    
    df: pd.DataFrame = df.rename(columns=fields)[fields.values()]
    
    df['主力资金净流入'] /= 100000000
    
    return df

df1 = get_zt_stock_bank()
df1

In [ ]:
import pandas as pd 
import numpy as np

df1 = pd.DataFrame({'depth': [0.500000,0.600000,1.300000],'VAR1': [38.196202,38.198002,38.200001],'profile': ['profile_1','profile_1','profile_1']})
df2 = pd.DataFrame({'depth': [0.600000,1.100000,1.200000],'VAR2': [0.20440,0.20442,0.20446]})
df3 = pd.DataFrame({'depth': [1.200000,1.300000,1.400000],'VAR3': [15.1880,15.1820,15.1820]})
df1
df1

In [ ]:
# dfmi['one']['second']
dfmi.loc[:, ('one', 'second')]

In [ ]:

import pandas as pd
import numpy as np

idx = pd.MultiIndex.from_arrays([
    ['warm', 'warm', 'cold', 'cold'],
    ['dog', 'falcon', 'fish', 'spider']],
    names=['blooded', 'animal'])
s = pd.Series([4, 2, 0, 8], name='legs', index=idx)
s

In [ ]:
import pandas as pd
from openpyxl.utils import get_column_letter
from pandas import ExcelWriter
import numpy as np
import sparklines

def to_excel_auto_column_weight(df: pd.DataFrame, writer: ExcelWriter, sheet_name):
    """DataFrame保存为excel并自动设置列宽"""
    df.to_excel(writer, sheet_name=sheet_name, index=False)
    #  计算表头的字符宽度
    column_widths = (
        df.columns.to_series().apply(lambda x: len(x.encode('gbk'))).values
    )
    #  计算每列的最大字符宽度
    max_widths = (
        df.astype(str).applymap(lambda x: len(x.encode('gbk'))).agg(max).values
    )
    # 计算整体最大宽度
    widths = np.max([column_widths, max_widths], axis=0)
    # 设置列宽
    worksheet = writer.sheets[sheet_name]
    for i, width in enumerate(widths, 1):
        # openpyxl引擎设置字符宽度时会缩水0.5左右个字符，所以干脆+2使左右都空出一个字宽。
        worksheet.column_dimensions[get_column_letter(i)].width = width + 2

df = pd.DataFrame({
    'Region': ['East', 'East', 'South', 'North', 'West', 'South', 'North', 'West', 'West', 'South', 'West', 'South'],
    'Item': ['Apple', 'Apple', 'Orange', 'Apple', 'Apple', 'Pear', 'Pear', 'Orange', 'Grape', 'Pear', 'Grape',
             'Orange'],
    'Volume1': [9000, 5000, 9000, 2000, 9000, 7000, 9000, 1000, 1000, 10000, 6000, 3000],
    'Volume2': [3000, 500, 2000, 1000, 8000, 7000, 6000, 700, 1000, 10000, 6000, 3000],
    'Month': ['July', 'July', 'September', 'November', 'November', 'October', 'August', 'December', 'November', 'April',
              'January', 'MaysdfasdfljasjASDFLAJSDFJADSFASFLASDFJASDLFJADSFKADSJF ASFDFDDSFASDFASD']
})

with pd.ExcelWriter('~/Downloads/123123.xlsx') as writer:
    to_excel_auto_column_weight(df, writer, 'TEST')

In [ ]:
import time
import pandas as pd
import openpyxl
from openpyxl.styles import Alignment
from openpyxl.chart.axis import DateAxis
from openpyxl.chart.label import DataLabelList
from openpyxl.chart import (
    LineChart,
    Reference,
)

# 简化起见，查询期间设置为年初至今天，Q4或12月适用，年初及其他情况需自行修改
today = time.strftime('%Y-%m-%d')
start_day = time.strftime('%Y') + '-01-01'

# 用pandas读取获取网页数据
url = 'http://www.safe.gov.cn/AppStructured/hlw/RMBQuery.do?startDate={start_day}&endDate={today}&queryYN=true'
df = pd.read_excel('~/Downloads/data.xls') #pd.read_html(url, encoding='utf-8')[4]   

# 文件保存，先将df存入excel，防止替换后日期列改变
save_dir = '/Users/zif/Downloads/rate1.xlsx'
writer = pd.ExcelWriter(save_dir)
df.to_excel(writer, '全部', index=None)

# 删除数据中的‘日’信息，仅截取年月信息以便按月汇总
df['日期'] = df['日期'].str.slice(0, 7)

# 获取指定币种的平均值
df1 = df.groupby(['日期'])[['美元', '欧元', '日元']].mean()

# 设置为各自常用汇率表达形式
df1['美元'] = round(df1['美元'] / 100, 2)  # 1美元兑n人民币
df1['欧元'] = round(df1['欧元'] / 100, 2)  # 1欧元兑n人民币
df1['日元'] = round(100 / df1['日元'], 2)  # 1人民币兑n日元

df1 = df1.reset_index()

# 将汇总表存入excel并保存
df1.to_excel(writer, '图表', index=None)
writer.save()

# 用openpyxl打开文件，修饰格式并创建图表
wb = openpyxl.load_workbook(save_dir)  # 加载既存工作表 #注意：这里是用的绝对路径，为什么不能用相对路径？总是报错

# 设置各工作表格式
names = wb.sheetnames
for name in names:
    ws = wb[name]
    irow = ws.max_row
    icol = ws.max_column
    ws.views.sheetView[0].showGridLines = False  # 设置不显示网格线
    ws.views.sheetView[0].zoomScale = 85  # 设置默认缩放比例
    # 仅对日期列设置宽度
    ws.column_dimensions['A'].width = 12
    # 设置使用区域居中显示
    align = Alignment(horizontal='center', vertical='center', wrap_text=True)
    for r in range(1, irow + 1):
        for c in range(1, icol + 1):
            ws.cell(r, c).alignment = align
    # 冻结拆分窗格，基点设为B2单元格
    ws.freeze_panes = 'B2'

# 将’图表‘sheet设置为活动工作簿，同时取消’全部‘sheet的选中状态
wb.active = wb['图表']
wb['全部'].views.sheetView[0].tabSelected = False

# 以下为创建折线图的内容，分日元、美元和欧元两段，每段代码基本相同，用不到可删掉其中一段

# 创建汇率折线图---日元
data = Reference(ws, min_col=4, min_row=1, max_col=4, max_row=13)
c1 = LineChart()
c1.title = "日元汇率推移图"
c1.style = 12
# 设置为不显示图例，不设置时默认显示图例
c1.legend = None
# 图例的位置可以通过设置其位置来控制：右、左、上、下和右上分别为r、l、t、b和tr。默认值为r
# c2.legend.position = 'tr'
# 设置纵轴名称
c1.y_axis.title = "汇率"
c1.y_axis.crossAx = 500
# 设置纵轴最大最小值和步长
c1.y_axis.scaling.min = 15
c1.y_axis.scaling.max = 19
c1.y_axis.majorUnit = 1
c1.x_axis = DateAxis(crossAx=100)
# c1.x_axis.title = "月"

# 自定义图表的宽度和高度，不设置时默认为15*7.5cm
c1.width = 25
c1.height = 10

c1.dLbls = DataLabelList()
# c2.dLbls.showCatName = True  # 数据标签显示x坐标轴标签
c1.dLbls.showVal = 1  # 将数据标签显示为数据对应的标签
# 设置标签位置，可选参数{'inBase', 'inEnd', 'l', 't', 'bestFit', 'outEnd', 'b', 'r', 'ctr'}
c1.dLbls.position = 't'  # 标签居上

c1.add_data(data, titles_from_data=True)
dates = Reference(ws, min_col=1, min_row=2, max_row=13)
c1.set_categories(dates)

# 设置系列1格式
s2 = c1.series[0]
# 设置线条颜色，不设置时默认为砖红色,其他颜色代码可参见 https://www.fontke.com/tool/rgb/00aaaa/
s2.graphicalProperties.line.solidFill = "FF5555"
s2.graphicalProperties.line.width = 30000  # 控制线条粗细
s2.smooth = True  # 设置为平滑曲线

# 插入图表
ws.add_chart(c1, "F2")

# 创建汇率折线图---美元欧元
# 数据区域为第2-3列，第2-13行，其他可自行调整
data = Reference(ws, min_col=2, min_row=1, max_col=3, max_row=13)
c2 = LineChart()
c2.title = "美元及欧元汇率推移图"
c2.style = 12
# 设置为不显示图例，默认为显示图例
# c2.legend=None
# 设置图例居上
c2.legend.position = 't'
# 坐标轴设置
c2.y_axis.title = "汇率"
c2.y_axis.crossAx = 500
# c2.y_axis.scaling.min = 5
# c2.y_axis.scaling.max = 9
c2.y_axis.majorUnit = 1
c2.x_axis = DateAxis(crossAx=100)
# c2.x_axis.title = "月"

# 自定义图表的宽度和高度，不设置时默认为15*7.5cm
c2.width = 25
c2.height = 12

c2.dLbls = DataLabelList()
# c2.dLbls.showCatName = True  # 数据标签显示x坐标轴标签
c2.dLbls.showVal = 1  # 将数据标签显示为数据对应的标签
# 设置标签居上
c2.dLbls.position = 't'

c2.add_data(data, titles_from_data=True)
dates = Reference(ws, min_col=1, min_row=2, max_row=13)
c2.set_categories(dates)

# 设置系列1格式
s1 = c2.series[0]
s1.graphicalProperties.line.width = 30000  # 控制线条粗细
s1.smooth = True  # 设置为平滑曲线

# 设置系列2格式
# s2 = c2.series[1]
# s2.graphicalProperties.line.solidFill = "00AAAA"  # 设置线条颜色
# s2.graphicalProperties.line.width = 30000  # 控制线条粗细
# s2.smooth = True  # 设置为平滑曲线

ws.add_chart(c2, "F28")

wb.save(save_dir)

In [ ]:
index = ['Ohio','Colorado','Utah','New York']
column = ['one','two','three','four']
df = pd.DataFrame(np.arange(16).reshape(4,4),index=index,columns=column)
df

In [ ]:
from pyecharts.charts import Bar
from pyecharts import options as opts

# V1 版本开始支持链式调用
bar = (
    Bar()
    .add_xaxis(["衬衫", "毛衣", "领带", "裤子", "风衣", "高跟鞋", "袜子"])
    .add_yaxis("商家A", [114, 55, 27, 101, 125, 27, 105])
    .add_yaxis("商家B", [57, 134, 137, 129, 145, 60, 49])
    .set_global_opts(title_opts=opts.TitleOpts(title="某商场销售情况"))
)
bar.render()

# 不习惯链式调用的开发者依旧可以单独调用方法
bar = Bar()
bar.add_xaxis(["衬衫", "毛衣", "领带", "裤子", "风衣", "高跟鞋", "袜子"])
bar.add_yaxis("商家A", [114, 55, 27, 101, 125, 27, 105])
bar.add_yaxis("商家B", [57, 134, 137, 129, 145, 60, 49])
bar.set_global_opts(title_opts=opts.TitleOpts(title="某商场销售情况"))
bar.render()

In [ ]:
from pyecharts.faker import Faker 
from pyecharts import options as opts 
from pyecharts.charts import Geo 
from pyecharts.globals import ChartType, SymbolType

In [ ]:
from pyecharts import Bar
from pyecharts.globals import CurrentConfig, NotebookType

CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

bar =Bar("我的第一个图表", "这里是副标题") #标题
bar.add("服装", #标签
  ["衬衫", "羊毛衫", "雪纺衫", "裤子", "高跟鞋", "袜子"],#x坐标
  [5, 20, 36, 10, 75, 90]) #y坐标
# is_more_utils = True #显示右侧的工具栏
# bar.show_config() # 打印所有参数
bar._render_notebook_component_()


In [4]:
import pandas as pd
import numpy as np

data1=pd.DataFrame({"score":[60,np.nan,75,80],"level":[np.nan,"a",np.nan,"f"],"coat":[1000,1500,np.nan,12000]})
data2=pd.DataFrame({"score":[34,58,np.nan],"level":[np.nan,"c","s"]})
data1.combine_first(data2)

,coat,level,score
0,1000.0,NaN,60.0
1,1500.0,a,58.0
2,NaN,s,75.0
3,12000.0,f,80.0


In [31]:
df =  pd.read_clipboard()
df

,所属行业,A,B,C,D
0,通用设备,7,16.0,116,17.0
1,电网设备,7,15.0,91,11.0
2,光伏设备,7,12.0,20,5.0
3,装修装饰,7,3.0,30,2.0
4,风电设备,6,9.0,4,NaN
5,铁路公路,6,5.0,19,NaN
6,专用设备,5,15.0,160,30.0
7,装修建材,4,4.0,51,7.0
8,工程建设,4,1.0,50,16.0
9,房地产开发,3,7.0,89,6.0


In [ ]:
import numpy as np
import pandas as pd